CLIP gives the similarity score of the image and text

In [19]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

url = "sample_image3.jpeg"
image = Image.open(url)

inputs = processor(text=["Name is Ram kumar and DOB is 17-06-1995 and Gender is Male and Number is 846550732129", "Name is Rakesh Kumar"], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities


In [20]:
probs

tensor([[0.9768, 0.0232]], grad_fn=<SoftmaxBackward0>)

In [27]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Load the image
url = "sample_image3.jpeg"
image = Image.open(url)

# Define your prompt (guide for the caption)
prompt = "What is the name in the document?"

# Process the image and generate a caption
inputs = processor(image, text=prompt, return_tensors="pt")
output = model.generate(**inputs)

# Decode the generated text (caption)
caption = processor.decode(output[0], skip_special_tokens=True)


/home/user/Meet_Patel/new1/newVenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/user/Meet_Patel/new1/newVenv/lib/python3.12/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [28]:
caption

'what is the name in the document?'

In [43]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
import torch

model_id = "google/paligemma-3b-mix-448"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id).eval()
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

/home/user/Meet_Patel/new1/newVenv/lib/python3.12/site-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [82]:
image = Image.open("pan.jpeg").convert("RGB")
# Instruct the model to create a caption in Spanish
# prompt = "In this document name is Adarsh kumar, dob is 17-06-1995, gender is male and aadhar number is 846550732129?"
# prompt = "In this document name is DUKHIYA PRASHAD PRAJAPATI, dob is 02-06-1990, gender is male and pan number is BJP24?"
prompt = "In this document which indian language are mentioned?"
# prompt = "What is name, DOB, Gender and AAdharnumber in this Indian Document?"
model_inputs = processor(text=prompt, images=image, return_tensors="pt")
input_len = model_inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]
    decoded = processor.decode(generation, skip_special_tokens=True)
    print(decoded)

hindi


In [86]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cpu" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")


/home/user/Meet_Patel/new1/newVenv/lib/python3.12/site-packages/accelerate/utils/modeling.py:1405: UserWarning: Current model requires 469765632 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [87]:

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [88]:
response

"A Large Language Model (LLM) is a type of artificial intelligence system designed to generate human-like text. These models are trained on vast amounts of textual data, enabling them to learn patterns and structures in language. LLMs can produce a wide range of outputs, from writing stories and scripts to summarizing documents and answering questions.\n\nThe training process for an LLM involves feeding it large datasets of text, such as books, articles, and web pages, so that it can learn the statistical relationships between words and their context. This allows the model to understand not just individual words, but also how they are used in sentences and paragraphs.\n\nOne key feature of LLMs is their ability to generate coherent and contextually relevant text by predicting the next word in a sequence based on the preceding words. They can also be fine-tuned for specific tasks or domains after initial training, making them versatile tools for various applications like chatbots, conte